In [1]:
pip install pandas psycopg2-binary openpyxl

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 2.7/2.7 MB 26.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    port=5437,
    database="db_maisondudroit",
    user="pgis",
    password="pgis"
)

cur = conn.cursor()


In [4]:
df = pd.read_excel("H:/SD3/SAE_DUBOIS_SABINE/Maison_droit_decembre.xlsx")


In [5]:
display(df)

,NUM,DATE_ENT,MODE_ENT,DUREE,SEXE,AGE,VIENT_PR,SIT_FAM,ENFANT,MODELE_FAM,...,Dem.2,Dem.3,Tot.dem,Sol.1,Sol.2,Sol.3,Tot.Sol,ORIGINE,COMMUNE,PARTENAIRE
0,3704,NaN,1,2,1,4,1,5a,0,NaN,...,3e,NaN,NaN,1,NaN,NaN,NaN,4e,St Avé,NaN
1,3705,NaN,5,1,4,3,6,3,1,1.0,...,1b,NaN,NaN,1,NaN,NaN,NaN,1b,Vannes,NaN
2,3706,NaN,5,1,1,4,1,4,0,NaN,...,7e,NaN,NaN,1,NaN,NaN,NaN,2a,Vannes K,NaN
3,3707,NaN,1,3,2,3,1,2,1,1.0,...,5b,NaN,NaN,1,2c,NaN,NaN,1b,Vannes M,NaN
4,3708,NaN,1,3,2,3,1,4,2,1.0,...,5b,NaN,NaN,1,2c,NaN,NaN,1b,Vannes M,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386,4091,NaN,5,2,2,3,1,7,NaN,NaN,...,NaN,NaN,NaN,2c,NaN,NaN,NaN,1b,Vannes M,Défenseur des Droit
387,4092,NaN,5,2,2,3,1,7,NaN,NaN,...,NaN,NaN,NaN,2c,NaN,NaN,NaN,1b,Vannes M,Défenseur des Droit
388,4093,NaN,5,2,2,3,1,7,NaN,NaN,...,NaN,NaN,NaN,2c,NaN,NaN,NaN,1b,Elven,Défenseur des Droit
389,4094,NaN,5,2,2,3,1,7,NaN,NaN,...,NaN,NaN,NaN,2c,NaN,NaN,NaN,1b,Ploeren,Défenseur des Droit


In [6]:
insert_entretien = """
INSERT INTO ENTRETIEN (
    NUM, DATE_ENT, MODE, DUREE, SEXE, AGE, VIENT_PR,
    SIT_FAM, ENFANT, MODELE_FAM, PROFESSION, RESS,
    ORIGINE, COMMUNE, PARTENAIRE
)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
"""


In [15]:
def to_int_or_none(val):
    if pd.isna(val):
        return None
    if isinstance(val, str) and val.lower() in ["nc"]:
        return None
    try:
        return int(val)
    except:
        return None

In [16]:
ok = 0

for i, row in df.iterrows():
    try:
        cur.execute(insert_entretien, (
            int(row["NUM"]),
            row["DATE_ENT"] if pd.notna(row["DATE_ENT"]) else None,
            to_int_or_none(row["MODE_ENT"]),
            to_int_or_none(row["DUREE"]),
            to_int_or_none(row["SEXE"]),
            to_int_or_none(row["AGE"]),
            to_int_or_none(row["VIENT_PR"]),
            row["SIT_FAM"],  # ← varchar(2)
            to_int_or_none(row["ENFANT"]),
            to_int_or_none(row["MODELE_FAM"]),
            to_int_or_none(row["PROFESSION"]),
            to_int_or_none(row["RESS"]),
            row["ORIGINE"],
            row["COMMUNE"],
            row["PARTENAIRE"]
        ))
        ok += 1
    except Exception as e:
        conn.rollback()
        print(" ERREUR à la ligne Excel :", i)
        print(row.to_dict())
        print(" PostgreSQL dit :", e)
        break

if ok > 0:
    conn.commit()
    print(f" {ok} entretiens insérés")

 391 entretiens insérés


In [22]:
conn.rollback()

In [23]:
insert_demande = """
INSERT INTO DEMANDE (num, pos, nature)
VALUES (%s, %s, %s)
"""

In [25]:
ok = 0
ignored = 0

for i, row in df.iterrows():
    num = int(row["NUM"])
    nature = row["Dem.1"]

    # on ignore vide / nc
    if pd.isna(nature) or (isinstance(nature, str) and nature.lower() == "nc"):
        ignored += 1
        continue

    try:
        cur.execute(insert_demande, (num, 1, str(nature)))
        ok += 1
    except Exception as e:
        conn.rollback()
        print("ERREUR DEMANDE 1")
        print("Ligne Excel :", i)
        print("num =", num, "nature =", nature)
        print("PostgreSQL dit :", e)
        raise e

conn.commit()

print(f"DEMANDE 1 : {ok} lignes insérées")
print(f"Ignorées : {ignored}")

❌ ERREUR DEMANDE 1
Ligne Excel : 0
num = 3704 nature = 3e
➡️ PostgreSQL dit : ERREUR:  la valeur d'une clé dupliquée rompt la contrainte unique « demande_pkey »
DETAIL:  La clé « (num, pos)=(3704, 1) » existe déjà.



UniqueViolation: ERREUR:  la valeur d'une clé dupliquée rompt la contrainte unique « demande_pkey »
DETAIL:  La clé « (num, pos)=(3704, 1) » existe déjà.


In [26]:
ok = 0
ignored = 0

for i, row in df.iterrows():
    num = int(row["NUM"])
    nature = row["Dem.2"]  # <-- on prend la colonne Dem.2

    # on ignore vide / nc
    if pd.isna(nature) or (isinstance(nature, str) and nature.lower() == "nc"):
        ignored += 1
        continue

    try:
        cur.execute(insert_demande, (num, 2, str(nature)))  # <-- pos = 2
        ok += 1
    except Exception as e:
        conn.rollback()
        print("ERREUR DEMANDE 2")
        print("Ligne Excel :", i)
        print("num =", num, "nature =", nature)
        print("PostgreSQL dit :", e)
        raise e

conn.commit()

print(f"DEMANDE 2 : {ok} lignes insérées")
print(f"Ignorées : {ignored}")

✅ DEMANDE 2 : 101 lignes insérées
ℹ️ Ignorées : 290


In [27]:
ok = 0
ignored = 0

for i, row in df.iterrows():
    num = int(row["NUM"])
    nature = row["Dem.3"]  # <- ici c'est Dem.3

    # on ignore vide / nc
    if pd.isna(nature) or (isinstance(nature, str) and nature.lower() == "nc"):
        ignored += 1
        continue

    try:
        cur.execute(insert_demande, (num, 3, str(nature)))  # <- pos = 3
        ok += 1
    except Exception as e:
        conn.rollback()
        print("ERREUR DEMANDE 3")
        print("Ligne Excel :", i)
        print("num =", num, "nature =", nature)
        print("PostgreSQL dit :", e)
        raise e

conn.commit()

print(f"DEMANDE 3 : {ok} lignes insérées")
print(f"Ignorées : {ignored}")

✅ DEMANDE 3 : 15 lignes insérées
ℹ️ Ignorées : 376


In [29]:
insert_solution = """
INSERT INTO SOLUTION (num, pos, nature)
VALUES (%s, %s, %s)
"""

ok = 0
ignored = 0

for i, row in df.iterrows():
    num = int(row["NUM"])
    nature = row["Sol.1"]

    # on ignore vide / nc
    if pd.isna(nature) or (isinstance(nature, str) and nature.lower() == "nc"):
        ignored += 1
        continue

    try:
        cur.execute(insert_solution, (num, 1, str(nature)))
        ok += 1
    except Exception as e:
        conn.rollback()
        print("ERREUR SOLUTION 1")
        print("Ligne Excel :", i)
        print("num =", num, "nature =", nature)
        print("PostgreSQL dit :", e)
        raise e

conn.commit()

print(f"SOLUTION 1 : {ok} lignes insérées")
print(f"Ignorées : {ignored}")


✅ SOLUTION 1 : 391 lignes insérées
ℹ️ Ignorées : 0


In [30]:
ok = 0
ignored = 0

for i, row in df.iterrows():
    num = int(row["NUM"])
    nature = row["Sol.2"]

    if pd.isna(nature) or (isinstance(nature, str) and nature.lower() == "nc"):
        ignored += 1
        continue

    try:
        cur.execute(insert_solution, (num, 2, str(nature)))
        ok += 1
    except Exception as e:
        conn.rollback()
        print("ERREUR SOLUTION 2")
        print("Ligne Excel :", i)
        print("num =", num, "nature =", nature)
        print("PostgreSQL dit :", e)
        raise e

conn.commit()

print(f"SOLUTION 2 : {ok} lignes insérées")
print(f"Ignorées : {ignored}")


✅ SOLUTION 2 : 112 lignes insérées
ℹ️ Ignorées : 279


In [31]:
ok = 0
ignored = 0

for i, row in df.iterrows():
    num = int(row["NUM"])
    nature = row["Sol.3"]

    if pd.isna(nature) or (isinstance(nature, str) and nature.lower() == "nc"):
        ignored += 1
        continue

    try:
        cur.execute(insert_solution, (num, 3, str(nature)))
        ok += 1
    except Exception as e:
        conn.rollback()
        print("ERREUR SOLUTION 3")
        print("Ligne Excel :", i)
        print("num =", num, "nature =", nature)
        print("PostgreSQL dit :", e)
        raise e

conn.commit()

print(f"SOLUTION 3 : {ok} lignes insérées")
print(f"Ignorées : {ignored}")


✅ SOLUTION 3 : 25 lignes insérées
ℹ️ Ignorées : 366


In [33]:
def export_table_to_excel(table_name, output_file):
    try:
        cur.execute(f"SELECT * FROM {table_name} ORDER BY num")  
        rows = cur.fetchall()
        colnames = [desc[0] for desc in cur.description]
        df = pd.DataFrame(rows, columns=colnames)
        df.to_excel(output_file, index=False)
        print(f"Export terminé : {output_file} ({len(df)} lignes)")
    except Exception as e:
        print(f"ERREUR lors de l'export de {table_name} :", e)

#Export des tables 
export_table_to_excel("ENTRETIEN", "entretien_export.xlsx")
export_table_to_excel("DEMANDE", "demande_export.xlsx")
export_table_to_excel("SOLUTION", "solution_export.xlsx")

✅ Export terminé : entretien_export.xlsx (391 lignes)
✅ Export terminé : demande_export.xlsx (507 lignes)
✅ Export terminé : solution_export.xlsx (528 lignes)
